# 뉴스 크롤링

In [ ]:
pip install selenium

In [ ]:
pip install webdriver-manager

In [ ]:
!apt-get update
!apt install chromium-chromedriver

In [152]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
import datetime
from pytz import timezone
KST = timezone('Asia/Seoul')

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=options)

## 기사 링크 크롤링

In [153]:
# 기사 링크 크롤링
# 카테고리당 100개 = 5페이지

url_list = []   # url 저장 리스트

category_list = []
categoty_name = ["정치", "경제", "사회", "생활/문화", "세계", "IT/과학"]

def getLink():    # 정치, 경제, 사회, 생활/문화, 세계, IT/과학
  a_list = []

  for category in range(6):     # 6
      for page in range(1, 6):  # 1, 6
          url = f'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={100 + category}#&date=%2000:00:00&page={page}'
          browser.get(url)

          time.sleep(1)

          soup = BeautifulSoup(browser.page_source, "html.parser")
          a_list.extend(soup.select(".type06_headline dt+dt a"))
          a_list.extend(soup.select(".type06 dt+dt a"))

          category_list.extend([categoty_name[category] for _ in range(20)])
          print(f"{categoty_name[category]} {page} 페이지")

  for a in a_list:
    url_list.append(a["href"])


def getEntertainmentLink():   # 연예
  today = str(datetime.datetime.now(KST))[:11]  # 서울 기준 시간
  a_list = []
  # today = datetime.date.today()   # 오늘 날짜 ex) 2023-10-31

  for page in range(1, 3):  # 1, 5
      url = f'https://entertain.naver.com/now#sid=106&date={today}&page={page}'
      browser.get(url)

      time.sleep(1)

      soup = BeautifulSoup(browser.page_source, "html.parser")

      a_list.extend(soup.select(".news_lst li>a"))

      category_list.extend(["연예" for _ in range(25)])
      print(f"연예 {page} 페이지")

  for a in a_list:
    url_list.append("https://entertain.naver.com" + a["href"])


def getSportsLink():    # 스포츠  (페이지마다 개수가 달라서 6페이지를 이동)
  a_list = []
  # today = str(datetime.date.today()).replace('-', '')  # 오늘 날짜 ex) 20231031
  today = str(datetime.datetime.now(KST))[:11].replace('-', '')  # 서울 기준 시간
  for page in range(1, 7):  # 1, 7
      url = f'https://sports.news.naver.com/general/news/index?isphoto=N&type=latest&date={today}&page={page}'
      browser.get(url)

      time.sleep(1)

      soup = BeautifulSoup(browser.page_source, "html.parser")
      a_list.extend(soup.select(".news_list li>a"))

      print(f"스포츠 {page} 페이지")

  for i in range(len(a_list)):
    if i == 100:  # 100개 링크 추가했으면 멈추기
      break
    url_list.append("https://sports.news.naver.com/news" + re.search('\?.+', a_list[i]["href"]).group())
    category_list.extend(["스포츠"])

# getLink()
getEntertainmentLink()
# getSportsLink()


연예 1 페이지
연예 2 페이지


In [154]:
print(len(url_list))
print(len(category_list))
print(url_list)

50
50
['https://entertain.naver.com/now/read?oid=213&aid=0001275216', 'https://entertain.naver.com/now/read?oid=003&aid=0012180763', 'https://entertain.naver.com/now/read?oid=408&aid=0000205259', 'https://entertain.naver.com/now/read?oid=241&aid=0003309646', 'https://entertain.naver.com/now/read?oid=117&aid=0003784031', 'https://entertain.naver.com/now/read?oid=213&aid=0001275215', 'https://entertain.naver.com/now/read?oid=003&aid=0012180761', 'https://entertain.naver.com/now/read?oid=382&aid=0001084742', 'https://entertain.naver.com/now/read?oid=382&aid=0001084741', 'https://entertain.naver.com/now/read?oid=382&aid=0001084740', 'https://entertain.naver.com/now/read?oid=438&aid=0000059857', 'https://entertain.naver.com/now/read?oid=312&aid=0000634660', 'https://entertain.naver.com/now/read?oid=438&aid=0000059856', 'https://entertain.naver.com/now/read?oid=108&aid=0003191637', 'https://entertain.naver.com/now/read?oid=117&aid=0003784030', 'https://entertain.naver.com/now/read?oid=609&ai

In [133]:
from pytz import timezone

KST = timezone('Asia/Seoul')
today = datetime.datetime.now(KST)  # 서울 기준 시간
print(str(today)[:11])
print(str(today)[:11].replace('-', ''))
today = datetime.date.today()       # 9시간 시차?
print(today)

2023-11-01 
20231101 
2023-10-31


In [155]:
dt = pd.DataFrame({"카테고리" : category_list,
                  "링크" : url_list})
dt

,카테고리,링크
0,연예,https://entertain.naver.com/now/read?oid=213&a...
1,연예,https://entertain.naver.com/now/read?oid=003&a...
2,연예,https://entertain.naver.com/now/read?oid=408&a...
3,연예,https://entertain.naver.com/now/read?oid=241&a...
4,연예,https://entertain.naver.com/now/read?oid=117&a...
5,연예,https://entertain.naver.com/now/read?oid=213&a...
6,연예,https://entertain.naver.com/now/read?oid=003&a...
7,연예,https://entertain.naver.com/now/read?oid=382&a...
8,연예,https://entertain.naver.com/now/read?oid=382&a...
9,연예,https://entertain.naver.com/now/read?oid=382&a...


## 제목, 본문, 날짜 크롤링

In [7]:
# 수집한 링크의 제목, 본문, 날짜 크롤링

title = []
content = []
date = []

for url in url_list:
      browser.get(url)

      time.sleep(0.5)

      soup = BeautifulSoup(browser.page_source, "html.parser")

      title.extend(soup.select("#title_area span"))                 # 제목

      c = soup.find_all(attrs={"id" : "dic_area"})                  # 본문

      while c[0].find(attrs={"class" : "end_photo_org"}):           # 이미지 있는 만큼
        c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기

      while c[0].find(attrs={"class" : "vod_player_wrap"}):         # 영상 있는 만큼
        c[0].find(attrs={"class" : "vod_player_wrap"}).decompose()  # 본문 영상에 있는 글자 없애기

      content.extend(c)

      date.extend(soup.select("._ARTICLE_DATE_TIME"))               # 날짜

## 전처리 or 제목, 본문, 날짜 리스트 생성

In [8]:
# 위에서 수집한 태그 속에서 내용만 뽑아서 담는다.

def clean(article):   # 전처리
  article = re.sub('\n','',article)
  article = re.sub('\t','',article)
  article = re.sub('\u200b','',article)
  article = re.sub('\xa0','',article)
  article = re.sub('([a-zA-Z])','',article)
  article = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',article)
  article = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',article)

  return article

title_list = []     # 제목
content_list = []   # 본문
date_list = []      # 날짜

for t in title:
  title_list.append(t.text)

for c in content:
  content_list.append(clean(c.text))

for d in date:
  date_list.append(d["data-date-time"])

## 데이터 프레임 생성

In [9]:
# 데이터 프레임으로 변환

article_df = pd.DataFrame({"카테고리" : category_list,
                           "날짜" : date_list,
                           "제목" : title_list,
                           "본문" : content_list,
                           "링크" : url_list})

article_df

,카테고리,날짜,제목,본문,링크
0,생활/문화,2023-10-31 21:16:01,"신안의 섬들, 김환기 추상화의 점이 되다",서해의 보석 신안 천사섬 11 세계 정상급 미학의 고향 안좌도신안군에는 1004개의...,https://n.news.naver.com/mnews/article/047/000...
1,생활/문화,2023-10-31 21:02:01,"""이젠 자장면·칼국수 사먹기도 겁난다""…먹거리 가격 '들썩'",주류업체들 잇달아 가격 올려햄버거 값도 도미노 인상서민 부담 가중 현실화“점심을 매...,https://n.news.naver.com/mnews/article/015/000...
2,생활/문화,2023-10-31 20:11:01,[날씨] 내일 중부 요란한 비…수도권 미세먼지 '나쁨',10월의 마지막 날인 오늘 수도권을 중심으로 공기가 탁한 곳이 많았습니다 이 먼지가...,https://n.news.naver.com/mnews/article/057/000...
3,생활/문화,2023-10-31 20:01:01,"나이 들어 약해진 악력, '이것' 먹었더니 강력!",60세 이상에서 근감소증 위험성도 낮아져단백질과 아미노산의 섭취가 악력 증가와 관련...,https://n.news.naver.com/mnews/article/296/000...
4,생활/문화,2023-10-31 19:46:01,[단독] “커피값 한잔에 무한 무료배달”…이 업체 ‘가격파괴’ 이유 있었네,비싸진 배달비에 앱이용 주춤하자 할인경쟁요기요 구독제 요기패스’ 4900원 반값’쿠...,https://n.news.naver.com/mnews/article/009/000...
5,생활/문화,2023-10-31 19:36:01,엄지윤만 뭇매? '전청조 패러디'가 낳은 '딜레마',개그맨 엄지윤이 전청조 패러디를 했다가 갑론을박에 휩싸였다 남현희 전 펜싱 국가대표...,https://n.news.naver.com/mnews/article/079/000...
6,생활/문화,2023-10-31 19:33:01,아이유와 이준기가 다녀간 ‘이곳’…中서 기술자만 80명 파견했다는데,최근 10대들 사이에서 마라탕후루’가 유행이다 식사로 중국 국물 요리 마라탕을 먹고...,https://n.news.naver.com/mnews/article/009/000...
7,생활/문화,2023-10-31 19:01:01,뱃살 빼는 데 좋은 ‘숨쉬기’ 방법 3,다이어트를 하는 사람들은 물만 마셔도 살이 찌는 것처럼 숨만 쉬어도 살이 빠지면 좋...,https://n.news.naver.com/mnews/article/346/000...
8,생활/문화,2023-10-31 18:59:01,[생활날씨] 내일 중부 곳곳 요란한 가을비…우박·강풍 주의,온화한 가을 마음껏 즐기고 계시나요10월의 마지막 날인 오늘도 예년 기온을 5도가량...,https://n.news.naver.com/mnews/article/422/000...
9,생활/문화,2023-10-31 18:52:31,[날씨] 내일도 '온화한 날씨'...수도권·영서 지방 가끔 비,11월의 첫날인 내일도 예년 기온을 크게 웃돌며 날이 온화하겠습니다 우선 서울 아침...,https://n.news.naver.com/mnews/article/052/000...


# 전처리

**KoNLPy**
- 한국어 정보처리를 위한 파이썬 패키지
- https://konlpy.org/ko/latest/api/konlpy.tag/#okt-class

In [10]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 29.2 MB/s eta 0:00:00


In [11]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# CountVectorizer : 텍스트 빈도만을 카운트해서 벡터화
# TfidfVectorizer : 자주 나오는 단어에 높은 가중치, 모든 문서에서 자주 나오는 단어에 패널티

In [12]:
okt = Okt()
nouns_list = []   # 명사 리스트

for content in article_df["본문"]:
  nouns_list.append(okt.nouns(content))     # 명사 추출, 리스트 반환

article_df["명사"] = nouns_list   # 데이터 프레임에 추가

In [13]:
text = [" ".join(noun) for noun in article_df["명사"]]  # 명사 열을 하나의 리스트에 담는다.

tfidf_vectorizer = TfidfVectorizer(min_df = 5, ngram_range=(1, 5))

# 군집화

In [14]:
from sklearn.cluster import DBSCAN
# DBSCAN : 밀도 기반 클러스터링
# https://hoonzi-text.tistory.com/19